# Tutorial on DER Hosting Capacity | <font color=red>Part 2</font>: Time-Series Analysis of Three-Phase Unbalanced LV Networks

Part 2 is about carrying out different time-series analyses of low voltage (LV) networks.

After doing this tutorial, you are expected to:
- Gain more understanding of the characteristics and behaviour of residential, three-phase LV distribution networks.
- Understand the simulation process necessary to study different solar PV penetration levels and assess the corresponding voltage rise and/or asset congestion issues.
- Understand how to determine the solar PV hosting capacity of LV networks.

## 1. Test LV Circuit and Considerations
The test LV circuit (also known as feeder) used in this tutorial is a realistic, anonymised residential three-phase LV circuit based on data provided by **[AusNet Services](https://www.ausnetservices.com.au/)**, who is one of the distribution companies (known as Distribution Network Service Providers [DNSPs] in Australia) in the State of Victoria, Australia.

The time-series active power values of residential load and PV generation used in this tutorial correspond to real, anonymised measurements from smart meters and dedicated PV generation meters that were also provided by **[AusNet Services](https://www.ausnetservices.com.au/)**.

- The test LV circuit is one of multiple circuits connected to the same transformer (this is why its size is relatively large). For simplicity, only one LV circuit is being considered.

- Also note that although the nominal line-to-line voltage is 400V on the secondary of the transformer, the transformer is actually having a transformation ratio that *boosts* the voltage on the secondary (around 8%). This *boost* is common practice in Australia and other countries around the world. Of course, the off-load tap changes can be used to tune the transformation ratio as needed.

- The topology is shown in **Figure 1**. There are 31 single-phase residential customers connected through single-phase service cables. As shown in the figure, these customers are connected to different phase groups: 11 to phase 1 (red), 10 to phase 2 (black) and 10 to phase 3 (blue). A summary of characteristics of this LV circuit are provided in **Table 1**. 

- The residential load dataset contains profiles for 342 customers for a whole year (365 days) in 30-minute intervals. In this particular case, it contains only active power (kW). Consequently, in this tutorial we will be creating the associated reactive power (kvar) profiles.

- The residential PV generation dataset contains daily normalised (from 0 to 1) profiles for a whole year in 30-minute intervals. Given that the sun shines simultaneously to everyone in a small area (such as an LV circuit), for the day being investigated, only one PV generation profile is selected and applied for all the PV systems. However, the actual PV generation depends on the size of each of the PV systems (and any PV inverter function being used).

- Once specific days to be studied are defined, the code randomly allocates profiles of demand to customers. Similarly, once a PV penetration is defined (percentage of customers with a PV system), the code randomly allocates PV systems to customers. However, we use a `seed` to keep the same random list for different PV penetrations.

- Given that the Test LV Circuit has small impedances due to its modern design, **the adopted PV systems are large (with 10 kVA inverters). This is simply to demonstrate voltage rise issues**. However, in Australia, it is common for distribution companies to impose a 5kW export limit to single-phase customers. In practice, this means that most installations have inverters of up to 5kVA.

<img style="float: middle;" src="LVcircuit-topology.png" width="60%">

**<center>Figure 1. Test LV Circuit Topology</center>**

| Customers (1-ph) | LV Lines/Segments (3-ph) | Service Cables (1-ph) | Distribution Transformer (3-ph) | Frequency |
| :------: | :------: | :------: | :------: | :------: |
| 31 | 24 | 31 | 22/0.433 kV/kV, 500 kVA (D-Y) | 50 Hz |

**<center>Table 1. Summary of characteristics of the Test LV Circuit</center>**

## 2. Initialisation
### 2.1 Import libraries
Run the code below to import the libraries that will be used in this tutorial.

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import math
import dss
from random import choice

### 2.2 Set the working path
Run the following code to set your working path.   
As an output you get the location in your computer.

In [ ]:
mydir = os.getcwd()
print("The files are located in the following path: mydir = %s" %mydir)

### 2.3 Set up  dss_engine
Before running the code, we need to set up the dss_engine.

In [ ]:
dss_engine = dss.DSS
DSSText = dss_engine.Text                                                      
DSSCircuit = dss_engine.ActiveCircuit                                            
DSSSolution = dss_engine.ActiveCircuit.Solution                                      
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue                                          
dss_engine.AllowForms = 0

## 3. Network, Load and DER Modelling
### 3.1 Define components
The `LVcircuit-master.txt` file defines the frequency and base voltages, then it redirects to network components definitions: transformers, lines, loads, etc.

In [ ]:
DSSText.Command = 'Clear'                               
DSSText.Command = 'Compile ' + mydir +  '\\LVcircuit-master.txt'    
DSSText.Command = 'Set VoltageBases = [22.0, 0.400]'
DSSText.Command = 'calcvoltagebases'

### 3.2 Load and PV profiles
#### 3.2.1 Import Load and PV profiles
The datasets used in this tutorial are:

- `Residential load data 30-min resolution.npy`: This numpy file contains the load profiles (only active power) for 342 customers for a whole year in 30-minute intervals.
- `Residential PV data 30-min resolution.npy`: This numpy file contains normalised PV generation profiles for a whole year in 30-minute intervals.

In [ ]:
houseData30minutes = np.load(mydir + '\\Residential load data 30-min resolution.npy') 
PVData30minutes = np.load(mydir + '\\Residential PV data 30-min resolution.npy')

#### 3.2.2 Store the *shape* of the Load and PV profiles  
Now that the data has been loaded, let's take a look at the dimensions of the numpy files.  
**Note:** The format of the house data is `(Customers, Days, Readings)` stored in `shape_profiles` and the PV profiles' data is `(Days, Readings)` stored in `PV_shape_profiles` which can be seen by running the code below.

In [ ]:
shape_profiles  = houseData30minutes.shape
noProfiles = shape_profiles[0]
PV_shape_profiles  = PVData30minutes.shape

print("The shape of the load profile is:", shape_profiles) 
print("The shape of the PV profiles is:", PV_shape_profiles) 

#### 3.2.3 Visualise the Load profiles

**Note:** For simplicity, the Load profiles of all customers for *only one day* given as `selected_day` are plotted below. Since the profiles in this example correspond to Australian data (i.e., Southern Hemisphere), the start of the year (January) ccorresponds to summer (Jan-Feb), then autumn (Mar-May), winter (Jun-Aug), spring (Sep-Nov), and summer again (Dec).

Feel free to change the value of the `selected_day` to get different load profiles for different days of the year.

In [ ]:
x=[i for i in range(48)]
l=[]
for i in range(24): 
      l.append("%s:00"%i)
      l.append(" ")
    
fig=plt.figure(figsize=(9,3)) 
selected_day=45                     #here is where you can change the selected day
for row in houseData30minutes[:,selected_day,:]:
   plt.plot(row, label = 'load profiles')
   plt.xlabel("Time (hh:mm)", fontsize = 15)
   plt.xticks(fontsize = 10)
   plt.xticks(x,l, fontsize=10, rotation=60)
   plt.ylabel('Active Power (kW)', fontsize = 15)
   plt.yticks(fontsize = 10)
   plt.title('Residential Load Profiles for the Selected Day', fontsize = 15)

#### 3.2.4 Visualise the normalised PV generation profiles
The plot of all normalised PV generation profiles for the whole year is shown by running the code below.

In [ ]:
fig=plt.figure(figsize=(9,3)) 
for row in PVData30minutes:
   plt.plot(row, label = 'pv profiles')
   plt.xlabel("Time (hh:mm)", fontsize = 15)
   plt.xticks(fontsize = 10)
   plt.xticks(x,l, fontsize=10, rotation=60)
   plt.ylabel('Normalised PV Generation', fontsize = 15)
   plt.yticks(fontsize = 10)
   plt.title('Residential Daily PV Generation Profiles in a Year', fontsize = 15)

### 3.3 Definition of Functions
#### 3.3.1 def <font color=blue> Load_profile_allocation</font> (New,icust,cust,iday)
- When `New`==1, generate a new loadShape with empty `load_profile`. Set power factor equal to 0.95
- When `New`==0, set **LoadShapes** equal to random `load_profile` of `iday` from `houseData30minutes`. Allocate list `keep_power_factor` as powerfactor
- Allocate the loadshape **customer_profile_icust** to **load.cust**.

In [ ]:
def Load_profile_allocation(New,icust,cust,iday):
    if New==1:
       load_profile = np.zeros(48)
       DSSCircuit.LoadShapes.New('customer_profile_%s' %(icust))
       DSSCircuit.LoadShapes.Npts = 48
       DSSCircuit.LoadShapes.MinInterval = 30
       DSSCircuit.LoadShapes.UseActual = 1
       DSSCircuit.LoadShapes.Qmult = (load_profile * math.tan(math.acos(0.95))).tolist()
    
    else:
        random_profile = np.random.randint(0,noProfiles)        
        load_profile = houseData30minutes[random_profile, iday, :]
        reactive_profile = []
        for j in range(48):
            reactive_profile.append((load_profile[j]) * math.tan(math.acos(keep_power_factor[icust][j])))
        reactive_profile = np.array(reactive_profile)          
        DSSCircuit.LoadShapes.Qmult = reactive_profile.tolist()
        
        
    DSSCircuit.LoadShapes.Name = 'customer_profile_%s' %(icust)
    DSSCircuit.LoadShapes.Pmult = load_profile.tolist()  
    DSSCircuit.SetActiveElement('load.%s'%(cust))
    DSSCircuit.ActiveElement.Properties('daily').Val = str('customer_profile_%s' %(icust))
    if New==0:
        return (load_profile,reactive_profile,random_profile)

#### 3.3.2 def <font color=blue> PV_allocation</font> (New,icust,cust,iday)
- When `New`==1, generate a loadshape with name **pv_profile_icust** and allocate empty `PV_profile` to it. Generate **PVSystem.icust** and allocate it to **load.cust**. Allocate the loadshape **pv_profile_icust** to **PVSystem.icust**.
- When `New`==0, allocate `PV_profile` of `iday` to the loadshape **pv_profile_icust**. Allocate the loadshape **pv_profile_icust** to **PVSystem.icust**. Allocate **PVSystem.icust** to **load.cust**.

In [ ]:
def PV_allocation(New,icust,cust,iday):
    if New==1:
        PV_profile = np.zeros(48)
        DSSCircuit.LoadShapes.New('pv_profile_%s' %(icust))
        DSSCircuit.LoadShapes.Npts = 48
        DSSCircuit.LoadShapes.MinInterval = 30
        DSSCircuit.LoadShapes.UseActual = 1
        DSSCircuit.LoadShapes.Pmult = PV_profile.tolist()
        DSSCircuit.SetActiveElement('load.%s' %(cust))
        bus_name = DSSCircuit.ActiveElement.Properties('bus1').Val
        DSSText.Command = 'new PVSystem.%s'%(icust)\
                                + ' phases = 1'  \
                                + ' irradiance = 1' \
                                + ' %cutin = 0.05' \
                                + ' %cutout = 0.05' \
                                + ' vmaxpu = 1.5' \
                                + ' vminpu = 0.5' \
                                + ' kva = 10' \
                                + ' pmpp = 10'\
                                + ' bus1 = ' + str(bus_name)\
                                + ' pf = 1' \
                                + ' kv = 0.23' \
                                + ' daily = ' +str('pv_profile_%s'%(icust))
    else:
        PV_profile = PVData30minutes[iday,:]
        pv_profiles_all.append(PV_profile)        
        DSSCircuit.LoadShapes.Name = 'pv_profile_%s' %(icust)
        DSSCircuit.LoadShapes.Pmult = PV_profile.tolist()        
        DSSCircuit.SetActiveElement('load.%s' %(cust))
        bus_name = DSSCircuit.ActiveElement.Properties('bus1').Val
        DSSCircuit.SetActiveElement('PVSystem.%s'%(icust))
        DSSCircuit.ActiveElement.Properties('bus1').Val = str(bus_name)
        DSSCircuit.ActiveElement.Properties('daily').Val = str('pv_profile_%s'%(icust))
        return PV_profile

#### 3.3.3 def <font color=blue>Initiate_variable</font> (customers)
The variables used to monitor different values (as explained below), are firstly initialised as empty:
- `kW_monitors`, `df_kw` will restore single phase active power of load. 
- `kvar_monitors`, `df_kvar` will restore single phase reactive power of load. 
- `voltages_monitors`, `df_volt` will restore single phase voltage of load. 
- `pd.DataFrame()` is used to restore data to excel. 
- `load_profiles_all` will restore all the `load_ profile`. 
- `pv_profiles_all` will resotre all the `pv_profile`. 
- `random_profiles_all` will restore all `random_profile`.
- `rand_cust` restore all the `random_customer`.
- `reactive_profiles_all` restore all the `reactive_profile`.


In [ ]:
def Initiate_variable(customers):
    kW_monitors = np.zeros((customers,48))
    kvar_monitors = np.zeros((customers,48))
    voltages_monitors = np.zeros((customers,48))
    df_kw = pd.DataFrame()
    df_kvar = pd.DataFrame()
    df_volt = pd.DataFrame()
    load_profiles_all = []
    pv_profiles_all = []
    random_profiles_all = []    
    rand_cust = []  
    reactive_profiles_all = []
    return  (kW_monitors, kvar_monitors, voltages_monitors, df_kw, df_kvar, 
             df_volt, load_profiles_all, pv_profiles_all, random_profiles_all, rand_cust,reactive_profiles_all)

#### 3.3.4 def <font color=blue>Monitors_data</font> ()

This function will be used to access and save active and reactive power as well as voltage magnitude of different load points. This is also a nice demonstration of accessing these measurements without the need of actually deploying monitors in our distribution network in OpenDSS.

In [ ]:
def Monitors_data():
    for icust, cust in enumerate(Loadname):
            DSSCircuit.SetActiveElement('load.%s' %(cust))
            kW_monitors[icust, itime] = DSSCircuit.ActiveElement.Powers[0]
            kvar_monitors[icust,itime]= DSSCircuit.ActiveElement.Powers[1]
            bus_name = DSSCircuit.ActiveElement.Properties('bus1').Val
            DSSCircuit.SetActiveBus(bus_name)
            voltages_monitors[icust, itime] = DSSCircuit.ActiveBus.puVmagAngle[0]

## 4. Simulation and Results
### 4.1  Initialise Load profiles and PV systems
After defining the functions, we are ready to allocate the Load and PV profiles

In [ ]:
Loadname = DSSCircuit.Loads.AllNames
for icust, cust in enumerate(Loadname):
    Load_profile_allocation(1,icust,cust,0)  
    PV_allocation(1,icust,cust,0)

### 4.2 Set simulation period
Set `start_day` and `total_day` to choose the simulation period you want.

In [ ]:
start_day = 0
total_days = 3

### 4.3 Set PV penetration level
Here we choose the percentage of PV penetration (0-100%) we want to investigate.
- PV penetration is defined here as the percentage of houses with PV systems. Given that we have 31 residential customers, a 50% PV penetration means 15 customers.

In [ ]:
penetration = 100
num_cust = len(Loadname)*(penetration/100)

### 4.4 Generate a random power factor list
Here a random power factor list is genereted a priori for each selected day (of the chosen `total days`) for all customers. This power factor will then be used to produce the associated reactive power profiles for each of the customers.

In [ ]:
keep_power_factor = []
customers = len(Loadname)
for i in range(total_days*customers):
    keep_pf = []
    for j in range(48):
        random_pf = random.uniform(0.9,0.98)
        keep_pf.append(random_pf)
    keep_power_factor.append(keep_pf)

### 4.5 Daily power flow analysis 
A 30-min-resolution power flow analysis is executed for the chosen period and specified penetration level. For each daily simulation, the load profiles and PV systems are randomly allocated. 

**Note:** `random.seed()` helps to generate the same random list, so you can change the number in the bracket to make a new one. The same random list is needed for keeping the same customers when increasing the PV penetration level.

See steps as below:
1. Initiate all the variables as introduced in the function `Initiate_variable()`. 

2. Use `Load_profile_allocation()` function to allocate load profiles to all loads.
3. Restore all the load names in `random_customer`. Shuffle `random_customer`. Restore `random_customer[numb]` to `rand_cust`.
4. Use `PV_allocation()` fucntion to allocate **PVSystem.numb** to **load.random_customer[numb]**.
5. Set OpenDSS solution mode to daily.
6. Run the simulation and check convergence.
7. Retrieve the results from dss_engine and plot voltage profiles.

In [ ]:
for iday in range(start_day,start_day+total_days):
    print("This is day: ",iday)
    np.random.seed(10)
    (kW_monitors, kvar_monitors, voltages_monitors, df_kw, df_kvar, df_volt, 
     load_profiles_all, pv_profiles_all, random_profiles_all, rand_cust,reactive_profiles_all) = Initiate_variable(customers)

    for icust, cust in enumerate(Loadname):
        load_profile,reactive_profile,random_profile=Load_profile_allocation(0,icust,cust,iday)
        load_profiles_all.append(load_profile)
        reactive_profiles_all.append(reactive_profile)
        random_profiles_all.append(random_profile)
        random.seed(90)
        
    random_customer = np.array(Loadname)
    random.shuffle(random_customer)    
    for numb in range(round(num_cust)):                
        rand_cust.append(choice([PV_cust for PV_cust in random_customer if PV_cust not in rand_cust]))
        PV_allocation(0,numb,rand_cust[numb],iday)
        PV_profile=PV_allocation(0,numb,rand_cust[numb],iday)
        pv_profiles_all.append(PV_profile)            
    DSSText.Command = 'Reset'       
    DSSText.Command = 'Set Mode=daily  number=1 stepsize=30m' 
        
    for itime in range(48):
        DSSSolution.Solve()
        Monitors_data()
    df_kw=pd.DataFrame(kW_monitors)
    df_kvar=pd.DataFrame(kvar_monitors)
    df_volt=pd.DataFrame(voltages_monitors)    
    
    if DSSSolution.Converged:
         print('Message: The Solution Converged Successfully\n')
    else:
         print("Message: The Solution Did Not Converge\n") 
    

        
    fig=plt.figure(figsize=(9,3)) 
    for idx, row in df_volt[0:31].iterrows():
             plt.plot(np.array(row), linewidth = 2)         
             plt.xlabel("Time (hh:mm)", fontsize = 15)
             plt.xticks(fontsize = 10)
             plt.xticks(x,l, fontsize=10, rotation=60)
             plt.ylabel('Voltage (p.u.)', fontsize = 15)
             plt.yticks(fontsize = 10)
             plt.title("Voltage Profile Day %s"%(iday+1), fontsize = 15)
             plt.ylim([1.05,1.13])
             plt.axhline(1.10, linewidth = 1,linestyle='dashed', color = 'r')
    plt.show()   

   

## 5. Further Analysis of the Results
### 5.1 Selected Load profiles & Customers allocated with **PV**

This section shows selected load profiles and customers which are allocated PV profiles. These are selected based on a random seed and then shuffled (**see section 4**) to include uncertainty in our analysis.

In [ ]:
print("Load profiles in this seed: ",random_profiles_all,customers,'\n')     
print("The customers with allocated PV profiles are:" ,rand_cust,'\n')

### 5.2 Verification of load curves

In this section, load curves are verified by plotting *actual load profile*, *monitored load* curves and *loadshapes* of **OpenDSS** loads. Actually, these are same load profiles but we want to verify if our code is using the same selected load profiles in **OpenDSS** simulation. For simplicity, only first load profile is verified.
If you want to check the load profile of other customer, change the `selected_customer` to another number (0 to 30).

In [ ]:
selected_customer=28
fig=plt.figure(figsize=(9,3))
plt.plot(load_profiles_all[selected_customer], label = 'actual')
plt.plot(kW_monitors[selected_customer,:], label = 'monitored', linestyle='-.')
DSSCircuit.LoadShapes.Name = 'customer_profile_%s' %(selected_customer)
plt.plot(DSSCircuit.LoadShapes.Pmult, label = 'loadshape_opendss', linestyle='dashed')
plt.xticks(x,l, fontsize=10, rotation=60)
plt.xlabel("Time (hh:mm)", fontsize = 15)
plt.ylabel('Active Power (kW)', fontsize = 15)
plt.title('Verification of Load Curves', fontsize = 15)
plt.legend()
plt.show()

### 5.3 Plot of selected PV profiles

This section plots **PV profiles** of the days that are being used in our simulation. 

In [ ]:
fig=plt.figure(figsize=(9,3))
plot_day=[i for i in range(start_day, total_days)]
for row in PVData30minutes[plot_day,:]:
   plt.plot(row, label = 'pv profiles')
   plt.xlabel("Time (hh:mm)", fontsize = 15)
   plt.xticks(fontsize = 10)
   plt.ylabel('Active Power (p.u.)', fontsize = 15)
   plt.xticks(x,l, fontsize=10, rotation=60)
   plt.yticks(fontsize = 10)
   plt.title('Daily PV Profiles', fontsize = 15)
plt.show()